In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('dataset_renamed.xlsx')

In [3]:
print(df.isnull().mean() * 100)

id                                                                    0.000000
target                                                               10.204082
Расстояние до метро                                                   0.000000
Суммарное кол-во отзывов в категории "АЗС" в радиусе 150м             0.000000
Суммарное кол-во отзывов в категории "Алкомаркеты" в радиусе 150м     0.000000
                                                                       ...    
Суммарный  трафик юзеров рабочих в радиусе 140м                       0.000000
Тип БЦ                                                                0.000000
Тип хаб                                                               0.000000
Тип улица                                                             0.000000
Тип ТЦ                                                                0.000000
Length: 2335, dtype: float64


In [4]:
print(df.isnull().mean() * 100)

id                                                                    0.000000
target                                                               10.204082
Расстояние до метро                                                   0.000000
Суммарное кол-во отзывов в категории "АЗС" в радиусе 150м             0.000000
Суммарное кол-во отзывов в категории "Алкомаркеты" в радиусе 150м     0.000000
                                                                       ...    
Суммарный  трафик юзеров рабочих в радиусе 140м                       0.000000
Тип БЦ                                                                0.000000
Тип хаб                                                               0.000000
Тип улица                                                             0.000000
Тип ТЦ                                                                0.000000
Length: 2335, dtype: float64
